In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import time
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib
from IPython.display import clear_output

In [2]:
plt.style.use('fivethirtyeight')
%matplotlib qt 
angsl = [0,0]
angsr = [0,0]
fig, [[ax1, ax2, ax3], [ax4,ax5,ax6]] = plt.subplots(2,3,figsize = (10,8))
#initiating mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles



#initiating constants
FILE_NAME = "D:/Career/GradProj/Datasets/Normal/person01_jogging_d1_uncomp.avi"
# VIDEO_NAME = "1_out"
N_TPOINTS = 6

#initiating video caputure
cap = cv2.VideoCapture(FILE_NAME)

# out = cv2.VideoWriter(VIDEO_NAME+".mp4",
# 		cv2.VideoWriter_fourcc(*"MJPG"),
# 		fps = 60,frameSize=(1920,1080))

#initiating records
pose_lms = [0]*32
record = []
frame = 0
plt.ion()
pos = []

In [3]:
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.4) as pose:
  while cap.isOpened():
    success, image = cap.read()
    try:
      image = cv2.resize(image, (160*8,120*8))
    except:
      break
    frame+=1
    if not success:
      break
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = True
    results = pose.process(image)
    landmark_list = []
    pose_lm = results.pose_landmarks
    if pose_lm:
      for j in range(len(pose_lm.landmark)):
        landmark_list.append([pose_lm.landmark[j].x,pose_lm.landmark[j].y,pose_lm.landmark[j].z])
    mp_drawing.draw_landmarks(image,
                              results.pose_landmarks,
                              mp_pose.POSE_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


    record.append(landmark_list)
    # Draw the pose annotation on the image.
    # image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    
    # cv2.imshow('MediaPipe Pose', image)
    image = cv2.resize(image, (1920//2,1080//2))
    if record[-1]:
        pos.append(record[-1])
        
    try:
      record[frame-1][24]
      hip = (np.array(record[frame-1][24])*np.array([1920//2,1080//2,1])).astype(int)
      knee = (np.array(record[frame-1][26])*np.array([1920//2,1080//2,1])).astype(int)
      ank = (np.array(record[frame-1][28])*np.array([1920//2,1080//2,1])).astype(int)

      ULF = hip[0:2] - knee[0:2]
      LLF = ank[0:2] - knee[0:2]
      LKA_NOM = np.sum(ULF*LLF)
      LKA_DENOM = np.linalg.norm(ULF)*np.linalg.norm(LLF)
      angsl.append(abs(180-np.arccos(LKA_NOM/LKA_DENOM)*180//pi))
      cv2.line(image,hip[0:2],knee[0:2],(0,255,0),5)
      cv2.line(image,knee[0:2],ank[0:2],(0,255,0),5)

      hip = (np.array(record[frame-1][23])*np.array([1920//2,1080//2,1])).astype(int)
      knee = (np.array(record[frame-1][25])*np.array([1920//2,1080//2,1])).astype(int)
      ank = (np.array(record[frame-1][27])*np.array([1920//2,1080//2,1])).astype(int)

      URF = hip[0:2] - knee[0:2]
      LRF = ank[0:2] - knee[0:2]
      RKA_NOM = np.sum(URF*LRF)
      RKA_DENOM = np.linalg.norm(URF)*np.linalg.norm(LRF)
      angsr.append(abs(180-np.arccos(RKA_NOM/RKA_DENOM)*180//pi))
      # cv2.line(image,[1920*(frame-1)//(415*2),angs[-2]],[1920*frame//(415*2),angs[-1]],(0,255,0),5)
      cv2.line(image,hip[0:2],knee[0:2],(255,0,0),5)
      cv2.line(image,knee[0:2],ank[0:2],(255,0,0),5)
      
      #center of hip
      COH = (np.asarray(pos)[:,24,0]+np.asarray(pos)[:,23,0])/2
      #y of anckle
      ank_yr = np.diff(np.asarray(pos)[:,27,1])
      ank_yl = np.diff(np.asarray(pos)[:,28,1])
      #center of hip velocity
      COH_v = abs(np.diff(COH))
    except:
      pass

    try:
      if pose_lm:
        ax1.plot(angsr, color = 'gold',label = 'right knee angle')
        ax1.plot(angsl, color = 'hotpink', label = 'left knee angle')
        ax1.set_title('right and left knee angle')

        ax2.plot(COH,color = 'gold', label= 'Center of hip position')
        ax2.set_title('Center of hip position')

        ax3.plot(COH_v,color = 'gold', label= 'Center of hip velocity')
        ax3.set_title('Center of hip velocity')
        
        ax4.plot(ank_yr,color = 'gold', label= 'anckle right y velocity')
        ax4.plot(ank_yl,color = 'hotpink', label= 'anckle left y velocity')
        ax4.set_title('anckle left y velocity')

        ax5.plot(abs(np.diff(angsr)),color = 'gold', label= 'right knee angle velocity')
        ax5.plot(abs(np.diff(angsl)),color = 'hotpink', label= 'left knee angle velocity')
        ax5.set_title('right and left knee angle velocity')
        
        fig.tight_layout()
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.pause(0.00001)
        # ax1.clear()
        # ax2.clear()
        # ax3.clear()
        # ax4.clear()
        # ax5.clear()
        # clear_output()
    except:
      pass
    
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break
cap.release()

In [4]:
plt.plot(np.asarray(pos)[:,23,0])

In [5]:
np.asarray(pos)[:,23,0]

array([ 9.43372130e-01,  9.13731873e-01,  8.94754052e-01,  8.70305896e-01,
        8.44759762e-01,  8.20814490e-01,  7.87348747e-01,  7.62448609e-01,
        7.32023656e-01,  7.04926848e-01,  6.73152030e-01,  6.44610345e-01,
        6.22131646e-01,  5.97256660e-01,  5.70136070e-01,  5.44707894e-01,
        5.14398217e-01,  4.85516459e-01,  4.53036219e-01,  4.29507822e-01,
        3.99143785e-01,  3.71241033e-01,  3.40363979e-01,  3.09875667e-01,
        2.86860675e-01,  2.61626959e-01,  2.38091141e-01,  2.09810212e-01,
        1.80224389e-01,  1.49399087e-01,  1.18589960e-01,  9.23276171e-02,
        6.12807646e-02,  1.88633967e-02,  7.78194575e-04, -3.54677849e-02,
        5.20977490e-02,  8.61000940e-02,  1.12304822e-01,  1.43094689e-01,
        1.77671790e-01,  2.02062324e-01,  2.33172581e-01,  2.55608171e-01,
        2.81026423e-01,  3.05835098e-01,  3.27265859e-01,  3.57556224e-01,
        3.85641396e-01,  4.18405592e-01,  4.49174106e-01,  4.78146970e-01,
        5.05056500e-01,  

In [6]:
len(record)

415

: 